## Week 2 Day 2

Our first Agentic Framework project!!

Prepare yourself for something ridiculously easy.

We're going to build a simple Agent system for generating cold sales outreach emails:
1. Agent workflow
2. Use of tools to call functions
3. Agent collaboration via Tools and Handoffs

## Before we start - some setup:


Please visit Sendgrid at: https://sendgrid.com/

(Sendgrid is a Twilio company for sending emails.)

If SendGrid gives you problems, see the alternative implementation using "Resend Email" in community_contributions/2_lab2_with_resend_email

Please set up an account - it's free! (at least, for me, right now).

Once you've created an account, click on:

Settings (left sidebar) >> API Keys >> Create API Key (button on top right)

Copy the key to the clipboard, then add a new line to your .env file:

`SENDGRID_API_KEY=xxxx`

And also, within SendGrid, go to:

Settings (left sidebar) >> Sender Authentication >> "Verify a Single Sender"  
and verify that your own email address is a real email address, so that SendGrid can send emails for you.


In [13]:
from dotenv import load_dotenv
from agents import FunctionTool
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio
import certifi
import os
import json



In [14]:
load_dotenv(override=True)

True

In [22]:
!uv pip install --upgrade certifi

2659.70s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Using Python 3.12.12 environment at: /Users/gonenc_aydin/Desktop/The_Complete_AI_Agent_Course/agents/.venv
Resolved 1 package in 186ms                                          
Audited 1 package in 0.34ms


In [27]:
# Let's just check emails are working for you



def send_test_email():
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("gonenc.aydin@intellecta.solutions")  # Must be verified in SendGrid
    to_email = To("gonencaydin@gmail.com")
    content = Content("text/plain", "This is an important test email")
    mail = Mail(from_email, to_email, "Test email", content)
    response = sg.client.mail.send.post(request_body=mail.get())
    print(response.status_code)
    print(response.body)
    print(response.headers)

send_test_email()

202
b''
Date: Tue, 18 Nov 2025 19:54:08 GMT
Content-Length: 0
Connection: close
Server: nginx
X-Message-Id: VxL74iN9TWe4bJj7N2XBDw
Access-Control-Allow-Origin: https://sendgrid.api-docs.io
Access-Control-Allow-Methods: POST
Access-Control-Allow-Headers: Authorization, Content-Type, On-behalf-of, x-sg-elas-acl
Access-Control-Max-Age: 600
X-No-CORS-Reason: https://sendgrid.com/docs/Classroom/Basics/API/cors.html
Strict-Transport-Security: max-age=31536000; includeSubDomains
Content-Security-Policy: frame-ancestors 'none'
Cache-Control: no-cache
X-Content-Type-Options: no-sniff
Referrer-Policy: strict-origin-when-cross-origin




In [52]:
import os
from sendgrid import SendGridAPIClient
from sendgrid.helpers.mail import Mail

def send_test_email():
    # Make sure SENDGRID_API_KEY is set in your environment
    sg = SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))

    message = Mail(
        from_email='gonenc.aydin@intellecta.solutions',  # Verified sender
        to_emails='gonencaydin@gmail.com',               # Recipient
        subject='Test Email',
        plain_text_content='This is an important test email'
    )

    try:
        response = sg.send(message)
        print("Status Code:", response.status_code)
        print("Headers:", response.headers)
        # 202 means accepted for delivery
        if response.status_code == 202:
            print("Email request accepted by SendGrid. Check inbox/spam.")
    except Exception as e:
        print("Error sending email:", e)

if __name__ == "__main__":
    send_test_email()

Status Code: 202
Headers: Date: Wed, 19 Nov 2025 00:00:57 GMT
Content-Length: 0
Connection: close
Server: nginx
X-Message-Id: 5S651aRoTcG-mMr0P1liyg
Access-Control-Allow-Origin: https://sendgrid.api-docs.io
Access-Control-Allow-Methods: POST
Access-Control-Allow-Headers: Authorization, Content-Type, On-behalf-of, x-sg-elas-acl
Access-Control-Max-Age: 600
X-No-CORS-Reason: https://sendgrid.com/docs/Classroom/Basics/API/cors.html
Strict-Transport-Security: max-age=31536000; includeSubDomains
Content-Security-Policy: frame-ancestors 'none'
Cache-Control: no-cache
X-Content-Type-Options: no-sniff
Referrer-Policy: strict-origin-when-cross-origin


Email request accepted by SendGrid. Check inbox/spam.


### Did you receive the test email

If you get a 202, then you're good to go!

#### Certificate error

If you get an error SSL: CERTIFICATE_VERIFY_FAILED then students Chris S and Oleksandr K have suggestions:  
First run this: `!uv pip install --upgrade certifi`  
Next, run this:
```python
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()
```

#### Other errors or no email

If there are other problems, you'll need to check your API key and your verified sender email address in the SendGrid dashboard

Or use the alternative implementation using "Resend Email" in community_contributions/2_lab2_with_resend_email

(Or - you could always replace the email sending code below with a Pushover call, or something to simply write to a flat file)

## Step 1: Agent workflow

In [53]:
instructions1 = "You are a Solutions Consultant working for Intellecta Solutions, \
a company that provides enterprise AI solutions, including DevSecOps automation, RAG-powered knowledge bases, and AI assistants for software development. \
Your role includes advising clients on technical roadmaps, proposing enterprise solutions, recommending best-fit tools, and architecting strategies for their AI and automation initiatives. \
Additionally, you craft professional cold emails to clients based on insights from their behaviors and interactions, helping guide them toward the best solutions for their needs."


instructions2 = "You are a humorous, engaging Solutions Consultant working for Intellecta Solutions, \
a company that provides enterprise AI solutions, including DevSecOps automation, RAG-powered knowledge bases, and AI assistants for software development. \
You advise clients on technical roadmaps, propose innovative enterprise solutions, recommend tools, and help architect AI-powered workflows. \
You also write witty, engaging cold emails to clients, tailored to their behaviors and interactions, that capture attention, entertain, and encourage them to explore your solutions."

instructions3 = "You are a busy Solutions Consultant working for Intellecta Solutions, \
a company that provides enterprise AI solutions, including DevSecOps automation, RAG-powered knowledge bases, and AI assistants for software development. \
You provide clients with technical roadmaps, propose enterprise solutions, recommend tools, and architect strategies for AI and automation adoption. \
You also write concise, actionable cold emails based on client behaviors and interactions, quickly communicating value and prompting engagement with your solutions."

In [54]:
solutions_agent1 = Agent(
        name="Professional Solutions Agent",
        instructions=instructions1,
        model="gpt-4o-mini"
)

solutions_agent2 = Agent(
        name="Engaging Solutions Agent",
        instructions=instructions2,
        model="gpt-4o-mini"
)

solutions_agent3 = Agent(
        name="Busy Solutions Agent",
        instructions=instructions3,
        model="gpt-4o-mini"
)


In [55]:
input_text = (
    "Write a professional cold email to a client who has previously shown interest "
    "in AI-powered DevOps and enterprise automation solutions. "
    "The email should introduce Intellecta Solutions, explain how our AI assistant "
    "and RAG-powered knowledge base can improve their development workflows, "
    "and suggest next steps for a demo or consultation."
)

result = Runner.run_streamed(solutions_agent1, input=input_text)
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)


Subject: Elevate Your Development Workflows with AI-Powered Solutions

Dear [Client's Name],

I hope this message finds you well. I’m [Your Name], and I represent Intellecta Solutions, where we specialize in enterprise AI solutions designed to transform and streamline development processes.

I noticed your previous interest in AI-powered DevOps and enterprise automation. At Intellecta Solutions, we harness the power of innovative AI technologies to enhance development workflows, ensuring that teams can achieve greater efficiency and collaboration.

Our AI Assistant is tailored to support developers by automating routine tasks, providing intelligent code suggestions, and facilitating seamless communication within teams. Complemented by our RAG-powered knowledge base, we empower your workforce to access real-time insights and relevant documentation, reducing time spent on searches and increasing productivity.

I would love to discuss how these tools can be tailored to meet your specific 

In [31]:
message = (
    "Write a cold email as a Solutions Consultant for Intellecta Solutions. "
    "Introduce our enterprise AI solutions including DevSecOps automation, RAG-powered knowledge bases, "
    "and AI assistants. Advise the client on potential solutions, technical roadmap, and next steps, "
    "tailored to their interests or behavior."
)

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(solutions_agent1, message),
        Runner.run(solutions_agent2, message),
        Runner.run(solutions_agent3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")





Subject: Unlocking the Future of Your Business with AI Solutions

Hi [Client’s Name],

I hope this message finds you well. I’m [Your Name], a Solutions Consultant at Intellecta Solutions, where we specialize in empowering organizations like yours with cutting-edge enterprise AI solutions.

As businesses increasingly embrace digital transformation, we understand that optimizing operations and enhancing security are top priorities. Our offerings include:

1. **DevSecOps Automation**: Streamline your software development lifecycle by integrating security protocols into your DevOps processes, ensuring faster releases without compromising on security.

2. **RAG-Powered Knowledge Bases**: Harness the power of Retrieval-Augmented Generation (RAG) to create intelligent knowledge bases that improve information accessibility and decision-making across your organization.

3. **AI Assistants for Software Development**: Enhance developer productivity with AI-driven assistants that provide code sugg

In [32]:
solution_evaluator = Agent(
    name="solution_evaluator",
    instructions="""
You are a Solutions Consultant evaluating email options. 
Pick the email that most effectively introduces enterprise AI solutions, DevOps automation, 
and AI-powered tools to the client, considering their behavior and interests. 
Do not explain your choice; reply with the selected email only.
""",
    model="gpt-4o-mini"
)

In [33]:
message = (
    "Write a cold email as a Solutions Consultant for Intellecta Solutions. "
    "Introduce our enterprise AI solutions including DevSecOps automation, RAG-powered knowledge bases, "
    "and AI assistants. Advise the client on potential solutions, technical roadmap, and next steps, "
    "tailored to their interests or behavior."
)

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(solutions_agent1, message),
        Runner.run(solutions_agent2, message),
        Runner.run(solutions_agent3, message),
    )
    outputs = [result.final_output for result in results]

    emails = "Cold solutions emails:\n\n" + "\n\nEmail:\n\n".join(outputs)

    best = await Runner.run(solution_evaluator, emails)

    print(f"Best solutions email:\n{best.final_output}")


Best solutions email:
Subject: Unlock New Possibilities with Intellecta Solutions' AI Innovations

Dear [Client's Name],

I hope this message finds you well. I'm [Your Name], a Solutions Consultant at Intellecta Solutions, where we specialize in empowering organizations like yours to harness the potential of enterprise AI.

In today’s fast-paced digital landscape, integrating cutting-edge solutions is vital for maintaining a competitive edge. Our offerings include:

- **DevSecOps Automation**: Streamline your development and security processes with our automated solutions, ensuring faster deliverables and enhanced security compliance.
  
- **RAG-Powered Knowledge Bases**: Elevate your decision-making with real-time, contextually relevant data at your fingertips, leveraging Retrieval-Augmented Generation for superior insights.

- **AI Assistants for Software Development**: Enhance productivity and efficiency within your development teams with AI-driven tools that assist in coding, debug

Now go and check out the trace:

https://platform.openai.com/traces

## Part 2: use of tools

Now we will add a tool to the mix.

Remember all that json boilerplate and the `handle_tool_calls()` function with the if logic..

In [56]:
solutions_agent1 = Agent(
        name="Professional Solutions Agent",
        instructions=instructions1,
        model="gpt-4o-mini"
)

solutions_agent2 = Agent(
        name="Engaging Solutions Agent",
        instructions=instructions2,
        model="gpt-4o-mini"
)

solutions_agent3 = Agent(
        name="Busy Solutions Agent",
        instructions=instructions3,
        model="gpt-4o-mini"
)

In [35]:
solutions_agent1

Agent(name='Professional Solutions Agent', instructions='You are a Solutions Consultant working for Intellecta Solutions, a company that provides enterprise AI solutions, including DevSecOps automation, RAG-powered knowledge bases, and AI assistants for software development. Your role includes advising clients on technical roadmaps, proposing enterprise solutions, recommending best-fit tools, and architecting strategies for their AI and automation initiatives. Additionally, you craft professional emails to clients based on insights from their behaviors and interactions, helping guide them toward the best solutions for their needs.', prompt=None, handoff_description=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extr

## Steps 2 and 3: Tools and Agent interactions

Remember all that boilerplate json?

Simply wrap your function with the decorator `@function_tool`

In [15]:
@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("gonenc.aydin@intellecta.solutions")  # Change to your verified sender
    to_email = To("gonencaydin@gmail.com")  # Change to your recipient
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Solutions email", content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

### This has automatically been converted into a tool, with the boilerplate json created

In [58]:
# Let's look at it
send_email

FunctionTool(name='send_email', description='Send out an email with the given body to all sales prospects', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x109ec0400>, strict_json_schema=True, is_enabled=True)

### And you can also convert an Agent into a tool

In [59]:
message = (
    "Write a cold email as a Solutions Consultant for Intellecta Solutions. "
    "Introduce our enterprise AI solutions including DevSecOps automation, RAG-powered knowledge bases, "
    "and AI assistants. Advise the client on potential solutions, technical roadmap, and next steps, "
    "tailored to their interests or behavior."
)
tool1 = solutions_agent1.as_tool(tool_name="solutions_agent1", tool_description=message)
tool1

FunctionTool(name='solutions_agent1', description='Write a cold email as a Solutions Consultant for Intellecta Solutions. Introduce our enterprise AI solutions including DevSecOps automation, RAG-powered knowledge bases, and AI assistants. Advise the client on potential solutions, technical roadmap, and next steps, tailored to their interests or behavior.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'solutions_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x109ef58a0>, strict_json_schema=True, is_enabled=True)

### So now we can gather all the tools together:

A tool for each of our 3 email-writing agents

And a tool for our function to send emails

In [39]:
# Function tool decorator ile tool tanımlama
@function_tool
def add_signoff(
    body: str,
    name: str = "Gönenç Aydin",
    title: str = "Solutions Consultant", 
    company: str = "Intellecta Solutions",
    contact: str = "gonenc.aydin@intellecta.solutions"
) -> str:
    """Append the consultant name, title, company, and contact to the email body"""
    
    signoff = f"""

Best regards,

{name}
{title}
{company}
{contact}"""
    
    return body + signoff

@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all solutions prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("gonenc.aydin@intellecta.solutions")  # Change to your verified sender
    to_email = To("gonencaydin@gmail.com")  # Change to your recipient
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Solutions email", content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}



In [30]:
solutions_agent1 = Agent(
    name="DevSecOps Specialist",
    instructions="""You are a Solutions Consultant at Intellecta Solutions specializing in DevSecOps automation.
    
    Write a compelling cold email (200-250 words) that:
    - Opens with a personalized hook about their DevOps challenges
    - Highlights our AI-powered security scanning that reduces vulnerabilities by 85%
    - Mentions our intelligent CI/CD pipelines that cut deployment time by 60%
    - Includes a specific case study (e.g., "We helped TechGiant Corp reduce security incidents by 90%")
    - Ends with a clear CTA for a 15-minute demo
    
    Focus on technical credibility and measurable ROI.""",
    model="gpt-4o-mini"
)

solutions_agent2 = Agent(
    name="RAG & Knowledge Specialist",
    instructions="""You are a Solutions Consultant at Intellecta Solutions specializing in RAG and knowledge management.
    
    Write an engaging cold email (200-250 words) that:
    - Starts with a pain point about information silos and knowledge loss
    - Showcases our RAG-powered knowledge bases with 95% accuracy in information retrieval
    - Emphasizes how we turn unstructured data into actionable insights
    - Mentions specific integrations (Confluence, SharePoint, Slack)
    - Includes ROI metrics (70% reduction in time spent searching for information)
    - Closes with an invitation for a personalized knowledge audit
    
    Emphasize productivity gains and competitive advantage.""",
    model="gpt-4o-mini"
)

solutions_agent3 = Agent(
    name="AI Assistant Specialist",
    instructions="""You are a Solutions Consultant at Intellecta Solutions specializing in AI assistants.
    
    Write a persuasive cold email (200-250 words) that:
    - Opens with a vision of automated workflows saving 30 hours per week
    - Highlights our custom AI assistants that integrate with existing tools
    - Mentions specific use cases (customer support automation, sales enablement, employee onboarding)
    - Includes adoption statistics (95% user satisfaction, 3x productivity increase)
    - References our white-glove implementation process
    - Ends with an offer for a free AI readiness assessment
    
    Focus on ease of adoption and immediate value.""",
    model="gpt-4o-mini"
)

# Tool descriptions
description = "Generate a professional cold email for Intellecta Solutions' enterprise AI offerings"

# Convert agents to tools
tool1 = solutions_agent1.as_tool(
    tool_name="devsecops_email_writer",
    tool_description=f"{description} - Focus on DevSecOps and security automation"
)
tool2 = solutions_agent2.as_tool(
    tool_name="rag_email_writer",
    tool_description=f"{description} - Focus on RAG and knowledge management"
)
tool3 = solutions_agent3.as_tool(
    tool_name="ai_assistant_email_writer",
    tool_description=f"{description} - Focus on AI assistants and automation"
)

# All tools
tools = [tool1, tool2, tool3, add_signoff, send_email]

## And now it's time for our Solutions Manager - our planning agent

In [31]:
# Solutions Manager instructions
instructions = """
You are a Solutions Manager at Intellecta Solutions orchestrating the email creation workflow.

Your workflow MUST follow these exact steps:

### Phase 1: Generate Email Drafts
You MUST call ALL THREE email writer tools to generate different perspectives:
1. Call `devsecops_email_writer` for a DevSecOps-focused email
2. Call `rag_email_writer` for a RAG/Knowledge-focused email  
3. Call `ai_assistant_email_writer` for an AI Assistant-focused email

### Phase 2: Evaluate and Select
After receiving all three drafts:
- Compare them based on:
  * Opening hook strength
  * Value proposition clarity
  * Specific metrics and case studies
  * Call-to-action effectiveness
  * Overall persuasiveness
- Select the SINGLE BEST email that would most likely get a positive response 

### Phase 3: Add Professional Sign-off
Call `add_signoff` with a JSON object containing:
```json
{
  "body": "[the selected best email content]",
  "name": "Gonench Aydin",
  "title": "Solutions Consultant",
  "company": "Intellecta Solutions",
  "contact": "gonench@intellecta.solutions"
}

###Phase 4: Send the Email
Call send_email with:

{
  "body": "[complete email with sign-off]",
  "to": "prospect@company.com",
  "subject": "Transform Your Enterprise with AI - Quick Question"
}



Important Rules:
- You MUST use the agent tools to generate drafts (don't write them yourself)
- You MUST select only ONE best email
- You MUST add sign-off using the add_signoff tool
- You MUST send only ONE final email
- Include specific technical details and business value in your selection criteria
"""

# Solutions Manager Agent'ı oluştur
solutions_manager = Agent(
    name="Solutions Manager",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini"
)


initial_message = {
        "role": "user",
        "content": """Please execute the complete email workflow for this prospect:

**Target Company:** TechCorp Industries
**Industry:** Financial Services
**Company Size:** 5,000 employees
**Current Challenges:**
- Security vulnerabilities increased by 40% last quarter
- Development cycles taking 3 weeks (industry average is 1 week)
- Customer data spread across 15 different systems
- Support team overwhelmed with 10,000+ tickets monthly
- Manual processes consuming 50% of employee time

**Recent News:** Just announced $50M funding for digital transformation

Please generate all 3 email drafts, select the best one based on their specific challenges, add the professional sign-off, and send the email.

Remember to use all the tools in the correct order."""
    }
    
try:
        print("🚀 Intellecta Solutions - Email Campaign System")
        print("="*60)
        print("Starting automated email workflow...")
        print("-"*60)
        
        # Run the workflow
        with trace("Email Campaign Workflow"):
            result = await Runner.run(
                solutions_manager,
                [initial_message],
                max_turns=12  # Enough turns for complete workflow
            )
        
        print("\n✅ Workflow completed successfully!")
        print("="*60)
        
        # Display results
        if result.final_output:
            print("\n📊 Final Result:")
            print(result.final_output)
        
        # Show execution summary
        if hasattr(result, 'messages'):
            tool_calls = []
            for msg in result.messages:
                if 'tool_calls' in msg:
                    for tc in msg['tool_calls']:
                        if isinstance(tc, dict) and 'name' in tc:
                            tool_calls.append(tc['name'])
            
            if tool_calls:
                print("\n📝 Tools Executed:")
                for i, tool in enumerate(tool_calls, 1):
                    print(f"  {i}. {tool}")
        
   
        
except Exception as e:
        print(f"\n❌ Error: {e}")
        import traceback
        traceback.print_exc()
        






🚀 Intellecta Solutions - Email Campaign System
Starting automated email workflow...
------------------------------------------------------------

✅ Workflow completed successfully!

📊 Final Result:
The email has been successfully sent to TechCorp Industries with the selected content focusing on DevSecOps and addressing their specific challenges. If you need any further assistance, feel free to ask!


<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Wait - you didn't get an email??</h2>
            <span style="color:#ff7800;">With much thanks to student Chris S. for describing his issue and fixes. 
            If you don't receive an email after running the prior cell, here are some things to check: <br/>
            First, check your Spam folder! Several students have missed that the emails arrived in Spam!<br/>Second, print(result) and see if you are receiving errors about SSL. 
            If you're receiving SSL errors, then please check out theses <a href="https://chatgpt.com/share/680620ec-3b30-8012-8c26-ca86693d0e3d">networking tips</a> and see the note in the next cell. Also look at the trace in OpenAI, and investigate on the SendGrid website, to hunt for clues. Let me know if I can help!
            </span>
        </td>
    </tr>
</table>

### And one more suggestion to send emails from student Oleksandr on Windows 11:

If you are getting certificate SSL errors, then:  
Run this in a terminal: `uv pip install --upgrade certifi`

Then run this code:
```python
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()
```

Thank you Oleksandr!

## Remember to check the trace

https://platform.openai.com/traces

And then check your email!!


### Handoffs represent a way an agent can delegate to an agent, passing control to it

Handoffs and Agents-as-tools are similar:

In both cases, an Agent can collaborate with another Agent

With tools, control passes back

With handoffs, control passes across



In [50]:

subject_instructions = "You can write a subject for a cold solutions email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold solutions email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")


In [51]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("gonenc.aydin@intellecta.solutions")  # Change to your verified sender
    to_email = To("gonencaydin@gmail.com")  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [52]:
tools = [subject_tool, html_tool, send_html_email]

In [53]:
tools

[FunctionTool(name='subject_writer', description='Write a subject for a cold solutions email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x116f0d940>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='html_converter', description='Convert a text email body to an HTML email body', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_converter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x116f0e160>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='send_html_email', description='Send out an email with the given subject and HTML body to all sales 

In [54]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")


### Now we have 3 tools and 1 handoff

In [55]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

[FunctionTool(name='devsecops_email_writer', description="Generate a professional cold email for Intellecta Solutions' enterprise AI offerings - Focus on DevSecOps and security automation", params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'devsecops_email_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x114bf3d80>, strict_json_schema=True, is_enabled=True), FunctionTool(name='rag_email_writer', description="Generate a professional cold email for Intellecta Solutions' enterprise AI offerings - Focus on RAG and knowledge management", params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'rag_email_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_t

In [56]:
# Improved instructions thanks to student Guillermo F.

solutions_manager_instructions = """
You are a Solutions Manager at Intellecta Solutions. Your goal is to find the single best cold solutions email using the solutions_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three solutions_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
 
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.
 
Crucial Rules:
- You must use the solutions agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""


solutions_manager = Agent(
    name="Solutions Manager",
    instructions=solutions_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold solutions email addressed to CEO"

with trace("Automated SDR"):
    result = await Runner.run(solutions_manager, message)

### Remember to check the trace

https://platform.openai.com/traces

And then check your email!!

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">Can you identify the Agentic design patterns that were used here?<br/>
            What is the 1 line that changed this from being an Agentic "workflow" to "agent" under Anthropic's definition?<br/>
            Try adding in more tools and Agents! You could have tools that handle the mail merge to send to a list.<br/><br/>
            HARD CHALLENGE: research how you can have SendGrid call a Callback webhook when a user replies to an email,
            Then have the SDR respond to keep the conversation going! This may require some "vibe coding" 😂
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">This is immediately applicable to Sales Automation; but more generally this could be applied to  end-to-end automation of any business process through conversations and tools. Think of ways you could apply an Agent solution
            like this in your day job.
            </span>
        </td>
    </tr>
</table>

## Extra note:

Google has released their Agent Development Kit (ADK). It's not yet got the traction of the other frameworks on this course, but it's getting some attention. It's interesting to note that it looks quite similar to OpenAI Agents SDK. To give you a preview, here's a peak at sample code from ADK:

```
root_agent = Agent(
    name="weather_time_agent",
    model="gemini-2.0-flash",
    description="Agent to answer questions about the time and weather in a city.",
    instruction="You are a helpful agent who can answer user questions about the time and weather in a city.",
    tools=[get_weather, get_current_time]
)
```

Well, that looks familiar!

And a student has contributed a customer care agent in community_contributions that uses ADK.